In [ ]:
! pip install synapseclient

In [ ]:
import pandas as pd
import numpy as np
import synapseclient as syna
from getpass import getpass


# Recorte solicitado pela Luciana em 21 de Janeiro de 2021


In [ ]:
 # Getting BD from Synapse
 syn = syna.Synapse()
 syn.login(input('Sybapse User: '), getpass('Passwd:'))

 # Obtain a pointer and download the data
# Banco_final.csv - https://www.synapse.org/#!Synapse:syn24225686
 dataset = syn.get(entity='syn24225686')

 # Get the path to the local copy of the data file
 filepath = dataset.path

Synapse passwd:··········
Welcome, Carlos Eduardo Beluzo!



In [ ]:
# Create a dataframe from the
banco_final = pd.read_csv(dataset.path, sep=";")
banco_final.head()

[WARNING] /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (693,694,700,701,702) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



,cod7_2019_g0,cod6_2019_g0,name_2019_g0,coduf_2019_g0,ufabv_2019_g0,ufname_2019_g0,pop_total_2010_g1,popUrbana_2010_g1,popRural_2010_g1,pop_total_2000_g1,popUrbana_2000_g1,popRural_2000_g1,popEstimada_2007,popEstimada_2008,popEstimada_2009,popEstimada_2011,popEstimada_2012,popEstimada_2013,popEstimada_2014,popEstimada_2015,popEstimada_2016,popEstimada_2017,popEstimada_2018,popEstimada_2019,popEstimada_2020,popProjetada_2010,popProjetada_2011,popProjetada_2012,popProjetada_2013,popProjetada_2014,popProjetada_2015,popProjetada_2016,popProjetada_2017,popProjetada_2018,popProjetada_2019,popProjetada_2020,popProjetada_2021,popProjetada_2022,popProjetada_2023,popProjetada_2024,...,esNivAtencao_hosp_media_complex_municipal_2009_g6,esNivAtencao_hosp_media_complex_municipal_2010_g6,esNivAtencao_hosp_media_complex_municipal_2011_g6,esNivAtencao_hosp_media_complex_municipal_2012_g6,esNivAtencao_hosp_media_complex_municipal_2013_g6,esNivAtencao_hosp_media_complex_municipal_2014_g6,esNivAtencao_hosp_media_complex_municipal_2015_g6,esNivAtencao_hosp_media_complex_municipal_2016_g6,esNivAtencao_hosp_media_complex_municipal_2017_g6,esNivAtencao_hosp_media_complex_municipal_2018_g6,esNivAtencao_hosp_media_complex_municipal_2019_g6,esNivAtencao_hosp_media_complex_municipal_2020_g6,esNivAtencao_hosp_alta_complex_estadual_2007_g6,esNivAtencao_hosp_alta_complex_estadual_2008_g6,esNivAtencao_hosp_alta_complex_estadual_2009_g6,esNivAtencao_hosp_alta_complex_estadual_2010_g6,esNivAtencao_hosp_alta_complex_estadual_2011_g6,esNivAtencao_hosp_alta_complex_estadual_2012_g6,esNivAtencao_hosp_alta_complex_estadual_2013_g6,esNivAtencao_hosp_alta_complex_estadual_2014_g6,esNivAtencao_hosp_alta_complex_estadual_2015_g6,esNivAtencao_hosp_alta_complex_estadual_2016_g6,esNivAtencao_hosp_alta_complex_estadual_2017_g6,esNivAtencao_hosp_alta_complex_estadual_2018_g6,esNivAtencao_hosp_alta_complex_estadual_2019_g6,esNivAtencao_hosp_alta_complex_estadual_2020_g6,esNivAtencao_hosp_alta_complex_municipal_2007_g6,esNivAtencao_hosp_alta_complex_municipal_2008_g6,esNivAtencao_hosp_alta_complex_municipal_2009_g6,esNivAtencao_hosp_alta_complex_municipal_2010_g6,esNivAtencao_hosp_alta_complex_municipal_2011_g6,esNivAtencao_hosp_alta_complex_municipal_2012_g6,esNivAtencao_hosp_alta_complex_municipal_2013_g6,esNivAtencao_hosp_alta_complex_municipal_2014_g6,esNivAtencao_hosp_alta_complex_municipal_2015_g6,esNivAtencao_hosp_alta_complex_municipal_2016_g6,esNivAtencao_hosp_alta_complex_municipal_2017_g6,esNivAtencao_hosp_alta_complex_municipal_2018_g6,esNivAtencao_hosp_alta_complex_municipal_2019_g6,esNivAtencao_hosp_alta_complex_municipal_2020_g6
0,1100015,110001,Alta Floresta D'oeste,11,RO,Rondônia,"24391,99999999981","0,5727287635290261","0,42727123647097381","26532,99999972","0,4651189085192113","0,53488109148078866",23857.0,24577.0,24354.0,24228.0,24069,25728,25652,25578,25506,25437,23167,22945,22728,"24947,53619498376","24675,27916551068","24401,88597555417","24127,59432670111","23857,50051652970","23590,61018510609","23288,210988509592","22986,155498966997","22688,2606900","22386,6326400","22081,0519300","21750,5380400","21417,601540","21081,879230","20743,011260",...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1100023,110002,Ariquemes,11,RO,Rondônia,"90352,99999999940","0,8469558288047979","0,15304417119520206","74503,00000123","0,7398091352060996","0,26019086479390047",82388.0,84581.0,85541.0,91570.0,92747,101269,102860,104401,105896,107345,106168,107863,109523,"92451,70372922070","94050,08588539537","95622,91370185446","97165,93159050276","98697,77390892536","100213,58679249488","101721,428258335611","103192,326299113920","104642,9362600","106029,4688100","107348,6927700","108673,5650600","109926,160050","111100,513850","112193,565820",...,9.0,11.0,11.0,11.0,11.0,11.0,11.0,10.0,10.0,11.0,11.0,12.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,N

In [ ]:
df[['cod7_2019_g0', 'cod6_2019_g0', 'name_2019_g0', 'coduf_2019_g0',
       'ufabv_2019_g0', 'ufname_2019_g0', 'pop_total_2010_g1',
       'popUrbana_2010_g1', 'popRural_2010_g1', 'pop_total_2000_g1',
       'popEstimada_2007', 'popEstimada_2008', 'popEstimada_2009', 'popEstimada_2011',
      'popEstimada_2012',	'popEstimada_2013',	'popEstimada_2014',
  'popEstimada_2015']].to_csv("banco_final_recorte1.csv")

# Criação de Store para códigos de municípios segundo IBGE
Usando HDFStore

In [ ]:
df = pd.read_csv("Lista_Estados_Brasil_Versao_CSV.csv", sep=";", encoding="ISO-8859-1")
estados = df[['IBGE','Estado','UF','Região']]
estados.columns=['cod_ibge','nome','sigla','regiao']
estados.head()

,cod_ibge,nome,sigla,regiao
0,11,Rondônia,RO,Região Norte
1,12,Acre,AC,Região Norte
2,13,Amazonas,AM,Região Norte
3,14,Roraima,RR,Região Norte
4,15,Pará,PA,Região Norte


In [ ]:
df = pd.read_csv("municipios_ibge_1.csv", sep=";", encoding="ISO-8859-1")
df[['IBGE','IBGE7','UF','Município']].to_csv("municipios_ibge.csv")
municipios = df[['IBGE','IBGE7','Município', 'UF']]
municipios.columns=['cod_ibge', 'cod_ibge7','nome', 'uf']
municipios.head()

,cod_ibge,cod_ibge7,nome,uf
0,110001,1100015,Alta Floresta D´oeste,RO
1,110002,1100023,Ariquemes,RO
2,110003,1100031,Cabixi,RO
3,110004,1100049,Cacoal,RO
4,110005,1100056,Cerejeiras,RO


In [ ]:
#!pip install --upgrade tables
store = pd.HDFStore('ibge.h5') # Create HDFS
# save it
store['estados'] = estados
store['municipios'] = municipios
store.close()

In [ ]:
# retrieve
store = pd.HDFStore('ibge.h5')
estados = store['estados']
municipios = store['municipios']

# Conversão do banco final em formato de série temporal

In [ ]:
# Criando registros para os ano de 2000 e período 2007 à 2030, para cada município

populacao = municipios[['cod_ibge7']].copy()
populacao['ano'] = 2000
ano = populacao.copy()

for i in range(2007,2031,1):
  populacao['ano'] = i
  ano = ano.append(populacao)

populacao = ano
populacao.reset_index(inplace=True)
populacao.drop(axis=0, columns={'index'}, inplace=True)

populacao.cod_ibge7.value_counts()

3162104    25
2916005    25
3302502    25
3509403    25
2925758    25
           ..
2515609    25
2919108    25
3101631    25
3148756    25
3143807    25
Name: cod_ibge7, Length: 5570, dtype: int64

In [ ]:
populacao.ano.value_counts()

2030    5570
2017    5570
2007    5570
2008    5570
2009    5570
2010    5570
2011    5570
2012    5570
2013    5570
2014    5570
2015    5570
2016    5570
2018    5570
2029    5570
2019    5570
2020    5570
2021    5570
2022    5570
2023    5570
2024    5570
2025    5570
2026    5570
2027    5570
2028    5570
2000    5570
Name: ano, dtype: int64

In [ ]:
for col in banco_final.columns:
  print(col)

In [ ]:
populacao['total'] = 0
populacao['urbana'] = 0
populacao['rural'] = 0

for index, row in banco_final.iterrows():
  cod_municipio = row.T.loc['cod7_2019_g0']

  valor = row.T.loc['pop_total_2000_g1']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2000)), 'total'] = valor

  valor = row.T.loc['pop_total_2010_g1']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2010)), 'total'] = valor

  valor = row.T.loc['popUrbana_2000_g1']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2000)), 'urbana'] = valor

  valor = row.T.loc['popUrbana_2010_g1']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2010)), 'urbana'] = valor

  valor = row.T.loc['popRural_2000_g1']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2000)), 'rural'] = valor

  valor = row.T.loc['popRural_2010_g1']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2010)), 'rural'] = valor

populacao.head()


,cod_ibge7,ano,total,urbana,rural
0,1100015,2000,"26532,99999972","0,4651189085192113","0,53488109148078866"
1,1100023,2000,"74503,00000123","0,7398091352060996","0,26019086479390047"
2,1100031,2000,"7517,99999981","0,3554136738570802","0,64458632614291989"
3,1100049,2000,"73568,00000040","0,6986461505019919","0,30135384949800803"
4,1100056,2000,"18207,00000003","0,8154006700651144","0,18459932993488562"


In [ ]:
populacao['estimada'] = 0

for index, row in banco_final.iterrows():
  cod_municipio = row.T.loc['cod7_2019_g0']

  valor = row.T.loc['popEstimada_2007']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2007)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2008']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2008)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2009']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2009)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2011']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2011)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2012']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2012)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2013']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2013)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2014']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2014)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2015']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2015)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2016']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2016)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2017']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2017)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2018']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2018)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2019']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2019)), 'estimada'] = valor

  valor = row.T.loc['popEstimada_2020']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2020)), 'estimada'] = valor

populacao[6000:6010]


,cod_ibge7,ano,total,urbana,rural,estimada,projetada
6000,1720002,2007,0,0,0,2291.0,0
6001,1720101,2007,0,0,0,4447.0,0
6002,1720150,2007,0,0,0,1409.0,0
6003,1720200,2007,0,0,0,10221.0,0
6004,1720259,2007,0,0,0,3012.0,0
6005,1720309,2007,0,0,0,4244.0,0
6006,1720499,2007,0,0,0,4885.0,0
6007,1720655,2007,0,0,0,5098.0,0
6008,1720804,2007,0,0,0,9302.0,0
6009,1720853,2007,0,0,0,1667.0,0


In [ ]:
populacao['projetada'] = 0

for index, row in banco_final.iterrows():
  cod_municipio = row.T.loc['cod7_2019_g0']

  valor = row.T.loc['popProjetada_2011']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2011)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2012']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2012)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2013']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2013)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2014']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2014)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2015']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2015)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2016']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2016)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2017']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2017)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2018']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2018)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2019']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2019)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2020']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2020)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2021']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2021)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2022']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2022)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2023']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2023)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2024']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2024)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2025']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2025)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2026']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2026)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2027']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2027)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2028']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2028)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2029']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2029)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2030']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2030)), 'projetada'] = valor

populacao[6000:6010]


,cod_ibge7,ano,total,urbana,rural,estimada,projetada
6000,1720002,2007,0,0,0,2291.0,0
6001,1720101,2007,0,0,0,4447.0,0
6002,1720150,2007,0,0,0,1409.0,0
6003,1720200,2007,0,0,0,10221.0,0
6004,1720259,2007,0,0,0,3012.0,0
6005,1720309,2007,0,0,0,4244.0,0
6006,1720499,2007,0,0,0,4885.0,0
6007,1720655,2007,0,0,0,5098.0,0
6008,1720804,2007,0,0,0,9302.0,0
6009,1720853,2007,0,0,0,1667.0,0


In [ ]:
populacao['projetada'] = 0

for index, row in banco_final.iterrows():
  cod_municipio = row.T.loc['cod7_2019_g0']

  valor = row.T.loc['popProjetada_2011']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2011)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2012']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2012)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2013']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2013)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2014']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2014)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2015']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2015)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2016']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2016)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2017']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2017)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2018']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2018)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2019']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2019)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2020']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2020)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2021']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2021)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2022']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2022)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2023']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2023)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2024']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2024)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2025']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2025)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2026']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2026)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2027']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2027)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2028']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2028)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2029']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2029)), 'projetada'] = valor

  valor = row.T.loc['popProjetada_2030']
  populacao.loc[((populacao.cod_ibge7 == cod_municipio) & (populacao.ano == 2030)), 'projetada'] = valor

populacao.head()


,cod_ibge7,ano,total
0,1100015,2000,0
1,1100023,2000,0
2,1100031,2000,0
3,1100049,2000,0
4,1100056,2000,0
...,...,...,...
139245,5222005,2030,0
139246,5222054,2030,0
139247,5222203,2030,0
139248,5222302,2030,0
